In [1]:
from pyspark import SparkContext
# sc = SparkContext.getOrCreate() 
sqlContext = SQLContext(sc)

In [2]:
import re
from PIL import Image
import os
import numpy as np
from tempfile import mkdtemp
import os.path as path
import matplotlib.pyplot as plt
import numpy as np
import theano
import keras
import traceback
from keras.models import Sequential
from scipy.misc import imread, imresize, imsave
from keras.layers.core import Flatten, Dense, Dropout,Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D,UpSampling2D,UpSampling1D,Cropping2D
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
%matplotlib inline
from keras import backend as K
from keras.utils import np_utils
K.set_image_dim_ordering('th')
import traceback
from scipy import ndimage
from sklearn.cross_validation import train_test_split
from keras import callbacks
import glob
from shutil import copyfile

Using TensorFlow backend.
/Users/home/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def imageResize(basename,imageName):
    """
    resize image
    basename : eg. /home/username/XYZFolder
    image name : xyz.jpg
    New folder in the working directory will be created with '_resized' as suffix
    """
    new_width  = 128
    new_height = 128
    try:  
        img = Image.open(basename+"/"+imageName) # image extension *.png,*.jpg
        img = img.resize((new_width, new_height), Image.ANTIALIAS)
        img.save(basename+'_resized/'+imageName)
    except:
        os.mkdir(basename+'_resized/')
        img = Image.open(basename+"/"+imageName) # image extension *.png,*.jpg
        img = img.resize((new_width, new_height), Image.ANTIALIAS)
        img.save(basename+'_resized/'+imageName)


In [3]:
def resizer(folderPath):
    """
    to resize all files present in a folder
    resizer('/home/username/XYZFolder')
    """
    
    for subdir, dirs, files in os.walk(folderPath):
        for fileName in files:
            try:
#                 print os.path.join(subdir, file)
                filepath = subdir + os.sep + fileName
#                 print filepath
                if filepath.endswith(".jpg" or ".jpeg" or ".png" or ".gif"):
                    imageResize(subdir,fileName)
            except:
                print traceback.print_exc()

In [4]:
# resizer('wiki_crop')

In [3]:
def load_image( infilename ) :
    img = ndimage.imread( infilename, flatten=True )
    data = np.asarray( img, dtype="int" )
#     print data.shape
    try :
        resized = data.reshape(data.shape[0],data.shape[1])
        return resized
    except :
        return 0

In [4]:
# some precautionary functions
def loadImageTest(fileName):
    """
    imdb database contain many images which are not in proper format, Due to format issue, fails to load raising an exception.
    To prevent such incidents, images are checked, if loads propely then only taken, else not considered
    """
    try:
        load_image(fileName)
        return 1
    except :
        return 0
    
def responseTest(diff):
    """
    all image names do not obey regex => r'nm*\d+_rm\d+_(\d+)-\d+-\d+_(\d+).jpg'
    possible reasons are as given below:
    1) no having proper format for naming
    2) due to naming error, sometime age are shown above 100 years, which is not valid for class size we have selected
    
    Images not satisfing age criterion are omitted
    """
    try:
        if diff > 0 and diff <= 100:
            return 1
    except:
        return 0

In [5]:
def turnToNumpy(files):
    """
    turn stored images on disk to numpy
    turnToNumpy('/home/username/XYZFolder/
    """
    temp = []
    y = []
    print "Total Count of Files",len(files)
    responseArray = []
    for fileName in files:
#                 try:
                    if fileName.endswith(".jpg" or ".jpeg" or ".png"):
                        
                        matchObj = re.search( r'\d+_(\d+)-\d+-\d+_(\d+).jpg', fileName, re.M|re.I)
#                         print matchObj
#                         print fileName
                        # yob - year of birth
                        # dopt - date of photo taken
                        yob =  matchObj.group(1)
                        dopt =  matchObj.group(2)
#                         print yob,dopt, int(dopt)-int(yob)
                        diff  = int(dopt)-int(yob)
                        if int(diff) > 0 and int(diff) <100:
#                             print type(int(dopt)-int(yob)),diff,diff > 0
#                             individualResponse = [0]*100
#                             individualResponse[diff] =1 
#                             responseArray.append(diff)
                            tmp = load_image(fileName)
                            if(not isinstance(tmp, int)):
                                temp.append((np.ravel(tmp), diff))
#                                 responseArray.append(diff)
#                             y.append(abs(int(dopt)-int(yob)))
#                 except:
#                     ""
    return temp

In [7]:
def myGeneratorEX(samples_per_epoch,epoch):
    
    folderName = "wiki_crop" # folder name where resized images are placed
    
    fileNames =  glob.glob(folderName+ '/*_resized'+"/*.jpg")#All file names with .jpg extension


    # first 100 imageswill be ued for onthe fly visual performace checking at each iteration
    initialFileNames = fileNames[:100]

    k =0 
    while k < epoch: # for each epoch do following
        print "Epoch : ",k," | Total Images : ",len(fileNames)
        for i in range(len(fileNames)/samples_per_epoch): 
            x_batch = turnToNumpy(fileNames[i*samples_per_epoch:(i+1)*samples_per_epoch])
            return x_batch        

In [8]:
x_batch = myGeneratorEX(10000,10)

Epoch :  0  | Total Images :  62328
Total Count of Files 10000


In [9]:
len(x_batch)

9652

In [10]:
len(x_batch[0][0])

16384

In [11]:
data_tmp = list()
for x in x_batch :
    data_tmp.append(([float(a) for a in x[0]], float(x[1])))
# data_tmp

In [12]:
data = sc.parallelize(data_tmp[0:20])

In [14]:
data.getNumPartitions()

4

In [13]:
data.collect()

[([210.0,
   211.0,
   205.0,
   202.0,
   213.0,
   228.0,
   222.0,
   205.0,
   214.0,
   222.0,
   217.0,
   212.0,
   220.0,
   222.0,
   218.0,
   220.0,
   226.0,
   216.0,
   202.0,
   214.0,
   198.0,
   138.0,
   84.0,
   38.0,
   24.0,
   16.0,
   13.0,
   18.0,
   16.0,
   9.0,
   8.0,
   13.0,
   16.0,
   22.0,
   31.0,
   41.0,
   46.0,
   46.0,
   41.0,
   38.0,
   32.0,
   47.0,
   40.0,
   42.0,
   55.0,
   42.0,
   44.0,
   86.0,
   65.0,
   44.0,
   64.0,
   67.0,
   49.0,
   59.0,
   58.0,
   38.0,
   46.0,
   82.0,
   68.0,
   78.0,
   88.0,
   86.0,
   91.0,
   42.0,
   55.0,
   64.0,
   73.0,
   73.0,
   63.0,
   49.0,
   39.0,
   35.0,
   35.0,
   39.0,
   41.0,
   36.0,
   28.0,
   25.0,
   31.0,
   38.0,
   21.0,
   50.0,
   64.0,
   72.0,
   98.0,
   113.0,
   94.0,
   69.0,
   46.0,
   48.0,
   41.0,
   11.0,
   33.0,
   47.0,
   78.0,
   89.0,
   77.0,
   37.0,
   46.0,
   67.0,
   55.0,
   67.0,
   103.0,
   112.0,
   171.0,
   213.0,
   212.0,
   211.0,
 

In [12]:
from pyspark.sql.types import *

# schema = StructType([
#   StructField("feature", ArrayType(IntegerType())),
#   StructField("label", IntegerType(), True)
# ])

In [15]:
from pyspark.sql import Row
# data_1 = data.map(lambda x : (x[0], int(x[1])))
image_df = sqlContext.createDataFrame(data.map(lambda x : Row(x[0], x[1])))
# data = data.map(lambda x : (list(x[0]), x[1]))

In [16]:
image_df.show()

+--------------------+----+
|                  _1|  _2|
+--------------------+----+
|[210.0, 211.0, 20...|67.0|
|[59.0, 54.0, 46.0...|27.0|
|[28.0, 28.0, 28.0...|46.0|
|[251.0, 245.0, 15...|43.0|
|[148.0, 148.0, 14...|48.0|
|[88.0, 88.0, 88.0...|38.0|
|[172.0, 155.0, 16...|29.0|
|[248.0, 253.0, 25...|25.0|
|[101.0, 118.0, 13...|68.0|
|[143.0, 143.0, 14...|60.0|
|[1.0, 0.0, 0.0, 0...|35.0|
|[74.0, 74.0, 74.0...|64.0|
|[244.0, 242.0, 24...|25.0|
|[96.0, 96.0, 96.0...|19.0|
|[23.0, 23.0, 32.0...|42.0|
|[193.0, 194.0, 19...|51.0|
|[105.0, 116.0, 12...|20.0|
|[48.0, 39.0, 39.0...|25.0|
|[0.0, 0.0, 0.0, 0...|33.0|
|[158.0, 158.0, 15...|31.0|
+--------------------+----+



In [17]:
sqlContext.registerDataFrameAsTable(image_df, "image_df_tab")

In [18]:
image_df = sqlContext.sql("SELECT _1 AS features, _2 as label from image_df_tab")
image_df.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[210.0, 211.0, 20...| 67.0|
|[59.0, 54.0, 46.0...| 27.0|
|[28.0, 28.0, 28.0...| 46.0|
|[251.0, 245.0, 15...| 43.0|
|[148.0, 148.0, 14...| 48.0|
|[88.0, 88.0, 88.0...| 38.0|
|[172.0, 155.0, 16...| 29.0|
|[248.0, 253.0, 25...| 25.0|
|[101.0, 118.0, 13...| 68.0|
|[143.0, 143.0, 14...| 60.0|
|[1.0, 0.0, 0.0, 0...| 35.0|
|[74.0, 74.0, 74.0...| 64.0|
|[244.0, 242.0, 24...| 25.0|
|[96.0, 96.0, 96.0...| 19.0|
|[23.0, 23.0, 32.0...| 42.0|
|[193.0, 194.0, 19...| 51.0|
|[105.0, 116.0, 12...| 20.0|
|[48.0, 39.0, 39.0...| 25.0|
|[0.0, 0.0, 0.0, 0...| 33.0|
|[158.0, 158.0, 15...| 31.0|
+--------------------+-----+



In [19]:
image_df.printSchema()

root
 |-- features: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- label: double (nullable = true)



In [20]:
df_train, df_test = image_df.randomSplit([0.8, 0.2])

In [21]:
df_train.cache()
df_test.cache()

DataFrame[features: array<double>, label: double]

In [22]:
df_train.is_cached

True

In [23]:
from pyspark.sql.functions import *
# from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors, VectorUDT

to_vector = udf(lambda a: Vectors.dense(a), VectorUDT())

train_temp = df_train.select(to_vector("features").alias("features"), "label")

In [24]:
train_temp.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[28.0,28.0,28.0,2...| 46.0|
|[59.0,54.0,46.0,4...| 27.0|
|[148.0,148.0,148....| 48.0|
|[251.0,245.0,154....| 43.0|
|[88.0,88.0,88.0,8...| 38.0|
|[172.0,155.0,164....| 29.0|
|[248.0,253.0,253....| 25.0|
|[1.0,0.0,0.0,0.0,...| 35.0|
|[23.0,23.0,32.0,3...| 42.0|
|[74.0,74.0,74.0,7...| 64.0|
|[96.0,96.0,96.0,9...| 19.0|
|[244.0,242.0,240....| 25.0|
|[48.0,39.0,39.0,4...| 25.0|
|[158.0,158.0,158....| 31.0|
|[193.0,194.0,199....| 51.0|
+--------------------+-----+



In [25]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(train_temp)

In [26]:
# save model
lrModel.save("lrmodelwith20")

In [32]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
test_temp = df_test.select(to_vector("features").alias("features"), "label")

lrpredicts = lrModel.transform(test_temp)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
accuracy = evaluator.evaluate(lrpredicts)
print "F1 = %.4f" %accuracy

F1 = 0.0000
